# 파일 처리기 구현
• 다양한 유형의 파일(텍스트, CSV, JSON, 바이너리)을 읽고 쓸 수 있어야 합니다

• 파일이 존재하지 않거나, 권한이 없거나, 형식이 잘못된 경우 등 다양한 오류 상황을 적절히 처리

• 사용자 정의 예외 계층 구조를 설계하고 구현

• 오류 발생 시 로깅을 통해 문제를 기록

• 모든 파일 작업은 컨텍스트 매니저(`with` 구문)를 사용

In [ ]:
# if, elif, else 구문을 통해서 다양한 유형의 파일 읽고 쓰기 형식처리
# 지원x 형식의 파일이 들어왔을 경우 예외처리 + '지원하지않는 형식입니다.' 출력.



In [ ]:
# 파일 입출력의 기본 패턴


file = open ('example.txt', 'r', encoding='utf-8')
# 파일 작업 수행
data = file.read()
print(data)
# 파일 작업 종료
file.close()

In [ ]:
# 텍스트 파일 읽기
with open('new.jpg', 'rb') as file:
    binary_data = file.read()
    print(binary_data)


# 텍스트 파일 쓰기
data = bytes([0x48, 0x65])

with open('binary_data.bin', 'wb') as file:
    file= file.write(data)

# CSV 파일 읽기

with open('data.csv', 'r', encoding='utf-8') as file:
    header = file.readline().strip().split(',')

    print(header)
    for line in file:
        values = line.strip().split(',')
        print(values)

# CSV 파일 쓰기
import csv
data = [
    ['Name', 'Age', 'City'],
    ['Alice', 30, 'New York'],
    ['Bob', 25, 'Los Angeles'],
    ['Charlie', 35, 'Chicago']
]
with open('output.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerows(data)
# JSON 파일 읽기
import json

with open('data.json', 'r', encoding='utf-8') as file:
    data = json.load(file)
    print(data)

# JSON 파일 쓰기
data = {
    'name': 'Alice',
    'age': 30,
    'city': 'New York'
}
with open('output.json', 'w', encoding='utf-8') as file:
    json.dump(data, file, ensure_ascii=False, indent=4)




로깅

In [ ]:
logging.basicConfig(
    level=logging.INFO
    format='%(asctime)s - %(levelname)s - %(message)s'
    filename='file_operations.log',
    encoding = 'utf'-8
    )


In [ ]:
import csv
import json
import logging
from pathlib import Path
from contextlib import contextmanager

# ---------------------------
# 로깅 설정
# ---------------------------
logging.basicConfig(
    filename="file_handler.log",
    level=logging.ERROR,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

# ---------------------------
# 사용자 정의 예외
# ---------------------------
class FileHandlerError(Exception):
    pass

class FileNotFoundErrorCustom(FileHandlerError):
    pass

class FilePermissionError(FileHandlerError):
    pass

class FileFormatError(FileHandlerError):
    pass

# ---------------------------
# FileHandler 클래스
# ---------------------------
class FileHandler:
    def __init__(self, filepath, filetype="text"):
        self.filepath = Path(filepath)
        self.filetype = filetype

    # ---------------------------
    # 컨텍스트 매니저
    # ---------------------------
    @contextmanager
    def open_reader(self):
        try:
            mode = "rb" if self.filetype == "binary" else "r"
            with open(self.filepath, mode, encoding=None if mode=="rb" else "utf-8") as f:
                yield f
        except FileNotFoundError:
            logging.error(f"파일 없음: {self.filepath}")
            raise FileNotFoundErrorCustom(f"{self.filepath} 파일을 찾을 수 없습니다")
        except PermissionError:
            logging.error(f"권한 오류: {self.filepath}")
            raise FilePermissionError(f"{self.filepath} 파일 접근 권한 없음")

    @contextmanager
    def open_writer(self):
        try:
            mode = "wb" if self.filetype == "binary" else "w"
            with open(self.filepath, mode, encoding=None if mode=="wb" else "utf-8") as f:
                yield f
        except PermissionError:
            logging.error(f"권한 오류: {self.filepath}")
            raise FilePermissionError(f"{self.filepath} 파일 접근 권한 없음")

    # ---------------------------
    # 파일 읽기
    # ---------------------------
    def read(self):
        with self.open_reader() as f:
            if self.filetype == "text":
                return f.read()
            elif self.filetype == "csv":
                return list(csv.reader(f))
            elif self.filetype == "json":
                return json.load(f)
            elif self.filetype == "binary":
                return f.read()
            else:
                raise FileFormatError(f"지원하지 않는 파일 형식: {self.filetype}")

    # ---------------------------
    # 파일 쓰기
    # ---------------------------
    def write(self, data):
        with self.open_writer() as f:
            if self.filetype == "text":
                f.write(str(data))
            elif self.filetype == "csv":
                writer = csv.writer(f)
                if isinstance(data, list):
                    writer.writerows(data)
                else:
                    raise FileFormatError("CSV 데이터는 리스트 형식이어야 합니다")
            elif self.filetype == "json":
                json.dump(data, f, ensure_ascii=False, indent=4)
            elif self.filetype == "binary":
                if isinstance(data, bytes):
                    f.write(data)
                else:
                    raise FileFormatError("바이너리 데이터는 bytes 형식이어야 합니다")
            else:
                raise FileFormatError(f"지원하지 않는 파일 형식: {self.filetype}")

# ---------------------------
# Step 1,2 연습용 실행
# ---------------------------
if __name__ == "__main__":
    # CSV 파일 읽기
    csv_handler = FileHandler("example.csv", "csv")
    data = csv_handler.read()
    print("CSV 읽기 결과:", data)

    # CSV 파일 쓰기 (새 파일 생성)
    csv_handler_out = FileHandler("example_out.csv", "csv")
    csv_handler_out.write([
        ["name", "age", "city"],
        ["David", 28, "Daegu"],
        ["Eve", 35, "Gwangju"]
    ])
    print("example_out.csv 파일 생성 완료")
